In [27]:
# idea:
# use 5 years data: 2016.1.1 - 2021.1.1 to train the model
# use 1 years data: 2020.1.1 - 2021.1.1 to find a good policy. 
# policy 1: choose the stock with maximum value, sell it when a) a stop loss/gain threshold is met.
# test it use 2021.1.1 to now.
import os
import pandas as pd
import json
from sklearn.linear_model import LinearRegression
import numpy as np

import matplotlib.pyplot as plt
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

In [28]:
def stack_table(df):
    df = df.drop(index=df.index[-1], axis=0)
    df_stack = df[['timestamp', 'open', 'close']].set_index('timestamp').stack()
    time_list = pd.Series([time + pd.Timedelta(hours=6.5) if time_type == 'close' else time for time, time_type in df_stack.index])
    df_stack = df_stack.reset_index().rename(columns={0: "price"}).drop(['timestamp', 'level_1'], axis=1)
    df_stack['timestamp'] = time_list
    df_stack['price_log'] = np.log(df_stack['price'])
    df_stack['price_log_diff'] = df_stack['price_log'].diff()
    df_stack.sort_values(by='timestamp', inplace=True)
    return df_stack

In [29]:
def add_value(df):
  df['value']=0
  for i in range(20):
    df['value']=df['price_log_diff'].shift(-1).fillna(0)+0.9*df['value'].shift(-1).fillna(0)
  return df

In [30]:
path = os.path.join('../data/stocks', 'AAPL')

In [31]:
df = pd.read_json(os.path.join(path, 'daily/data'))

In [32]:
df_stack = stack_table(df)

In [33]:
df = add_value(df_stack)

In [34]:
df

,price,timestamp,price_log,price_log_diff,value
0,0.386429,2002-06-11 13:30:00,-0.950807,NaN,-5.991296
10074,0.386429,2002-06-11 13:30:00,-0.950807,-5.871737,-0.133635
10075,0.365357,2002-06-11 20:00:00,-1.006880,-0.056073,-0.087978
1,0.365357,2002-06-11 20:00:00,-1.006880,-0.056073,-0.037248
2,0.364464,2002-06-12 13:30:00,-1.009327,-0.002447,-0.058722
...,...,...,...,...,...
10070,147.080002,2022-06-09 13:30:00,4.990977,-0.005965,-0.088310
20145,142.639999,2022-06-09 20:00:00,4.960324,-0.030653,-0.064064
10071,142.639999,2022-06-09 20:00:00,4.960324,-0.030653,-0.037123
10072,140.279999,2022-06-10 13:30:00,4.943640,-0.016684,-0.022711


In [35]:
rootdir = '../data/indicators'
indicators_list = []
for subdir, dirs, files in os.walk(rootdir):
  for dir in dirs:
    name = dir
    if '=' in name:
      indicators_list.append(name)

In [36]:
indicators_list


['EURSEK=X',
 'CL=F',
 'EUR=X',
 'ZB=F',
 'GC=F',
 'HG=F',
 'ZS=F',
 'SEK=X',
 'USDEUR=X',
 'LE=F',
 'SB=F',
 'SI=F']

In [37]:
def rename_dataframe(df, name):
  return df.rename(columns={'price': 'price_' + name,
                            'price_log': 'price_log_' + name,
                            'price_log_diff': 'price_log_diff_' + name
                            })

In [38]:
for indicator in indicators_list:
    print('merging name: ' + indicator)
    path = os.path.join('../data/indicators', indicator)
    df_ind = pd.read_json(os.path.join(path, 'daily', 'data')).dropna()
    df_stack_ind = stack_table(df_ind)
    # df_stack_ind
    df = rename_dataframe(df, indicator)
    df = pd.merge_asof(df, df_stack_ind, on='timestamp')


merging name: EURSEK=X
merging name: CL=F


/home/ken/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


merging name: EUR=X
merging name: ZB=F
merging name: GC=F
merging name: HG=F
merging name: ZS=F
merging name: SEK=X
merging name: USDEUR=X
merging name: LE=F
merging name: SB=F
merging name: SI=F


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20146 entries, 0 to 20145
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   price_EURSEK=X           20146 non-null  float64       
 1   timestamp                20146 non-null  datetime64[ns]
 2   price_log_EURSEK=X       20146 non-null  float64       
 3   price_log_diff_EURSEK=X  20145 non-null  float64       
 4   value                    20146 non-null  float64       
 5   price_CL=F               20146 non-null  float64       
 6   price_log_CL=F           20146 non-null  float64       
 7   price_log_diff_CL=F      20146 non-null  float64       
 8   price_EUR=X              20142 non-null  float64       
 9   price_log_EUR=X          20138 non-null  float64       
 10  price_log_diff_EUR=X     20134 non-null  float64       
 11  price_ZB=F               18658 non-null  float64       
 12  price_log_ZB=F           18658 n

In [40]:
def reorder(df):

  columns_non_value = [name for name in df.columns if name != 'value']
  df = df[columns_non_value + ['value']]
  return df

In [41]:
df = reorder(df)

In [42]:
df.corr()['value'].abs().sort_values(ascending=False)

value                      1.000000
price_log_diff_EURSEK=X    0.069789
price_log_diff_GC=F        0.068778
price_SEK=X                0.062239
price_log_SEK=X            0.058575
price_log_diff_ZS=F        0.046769
price_SB=F                 0.046441
price_log_diff_EUR=X       0.046157
price_log_SB=F             0.046108
price_log_diff_USDEUR=X    0.041113
price_EUR=X                0.040380
price_log_diff_SB=F        0.037768
price_log_EUR=X            0.031075
price                      0.030939
price_log_diff_SI=F        0.030415
price_log_HG=F             0.030247
price_log                  0.028895
price_HG=F                 0.028816
price_ZS=F                 0.028212
price_log_diff_CL=F        0.024964
price_log_diff_ZB=F        0.024418
price_log_diff_LE=F        0.024418
price_log_diff             0.023945
price_log_ZS=F             0.020435
price_log_diff_SEK=X       0.019954
price_CL=F                 0.019455
price_log_CL=F             0.019212
price_EURSEK=X             0

In [43]:
column_list = [name for name in df.columns if name != 'value' and name != 'timestamp']



In [44]:
column_list

['price_EURSEK=X',
 'price_log_EURSEK=X',
 'price_log_diff_EURSEK=X',
 'price_CL=F',
 'price_log_CL=F',
 'price_log_diff_CL=F',
 'price_EUR=X',
 'price_log_EUR=X',
 'price_log_diff_EUR=X',
 'price_ZB=F',
 'price_log_ZB=F',
 'price_log_diff_ZB=F',
 'price_GC=F',
 'price_log_GC=F',
 'price_log_diff_GC=F',
 'price_HG=F',
 'price_log_HG=F',
 'price_log_diff_HG=F',
 'price_ZS=F',
 'price_log_ZS=F',
 'price_log_diff_ZS=F',
 'price_SEK=X',
 'price_log_SEK=X',
 'price_log_diff_SEK=X',
 'price_USDEUR=X',
 'price_log_USDEUR=X',
 'price_log_diff_USDEUR=X',
 'price_LE=F',
 'price_log_LE=F',
 'price_log_diff_LE=F',
 'price_SB=F',
 'price_log_SB=F',
 'price_log_diff_SB=F',
 'price_SI=F',
 'price_log_SI=F',
 'price_log_diff_SI=F',
 'price',
 'price_log',
 'price_log_diff']

In [70]:
# Numerical pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

num_pipeline = Pipeline([ 
 ('imputer', SimpleImputer(strategy="mean")),
 ('std_scaler', StandardScaler() )])

num_pipeline_normalizer = Pipeline([ 
 ('imputer', SimpleImputer(strategy="mean")),
 ('normalizer', Normalizer() )])

# categorical pipeline
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([ 
 ('imputer', SimpleImputer(strategy="most_frequent")),
 ('OneHotEncoder', OneHotEncoder())
 ])


from sklearn.compose import ColumnTransformer

num_attribs = column_list
cat_attribs = []

# preprocess pipeline
from sklearn.compose import ColumnTransformer
column_pipeline = ColumnTransformer([("num", num_pipeline, num_attribs),("cat", cat_pipeline, cat_attribs)])
column_pipeline_normal = ColumnTransformer([("num", num_pipeline_normalizer, num_attribs),("cat", cat_pipeline, cat_attribs)])

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20146 entries, 0 to 20145
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   price_EURSEK=X           20146 non-null  float64       
 1   timestamp                20146 non-null  datetime64[ns]
 2   price_log_EURSEK=X       20146 non-null  float64       
 3   price_log_diff_EURSEK=X  20145 non-null  float64       
 4   price_CL=F               20146 non-null  float64       
 5   price_log_CL=F           20146 non-null  float64       
 6   price_log_diff_CL=F      20146 non-null  float64       
 7   price_EUR=X              20142 non-null  float64       
 8   price_log_EUR=X          20138 non-null  float64       
 9   price_log_diff_EUR=X     20134 non-null  float64       
 10  price_ZB=F               18658 non-null  float64       
 11  price_log_ZB=F           18658 non-null  float64       
 12  price_log_diff_ZB=F      18658 n

In [72]:
len(column_list)

39

In [73]:
#Train Data: From 2013-01-01 - 2021-01-01
X, y = df.iloc[:7000, 0:-1], df.iloc[:7000, -1]

In [74]:
column_pipeline.fit_transform(X)

array([[-1.00971307e+00, -1.21417778e+00,  7.43533391e-19, ...,
         8.71068260e-16,  0.00000000e+00,  1.99560207e-18],
       [-1.00971307e+00, -1.21417778e+00, -8.05334464e+01, ...,
         8.71068260e-16,  0.00000000e+00,  1.99560207e-18],
       [-1.02048168e+00, -1.26544156e+00, -7.67144849e-01, ...,
         8.71068260e-16,  0.00000000e+00,  1.99560207e-18],
       ...,
       [ 1.07696878e+00,  1.02396218e+00, -7.67387415e-02, ...,
         1.19915098e+00,  1.15584862e+00,  3.96363706e-01],
       [ 1.05926476e+00,  1.01684859e+00, -1.04779727e-01, ...,
         1.19915098e+00,  1.15584862e+00,  3.96363706e-01],
       [ 1.05926476e+00,  1.01684859e+00, -1.04779727e-01, ...,
         1.19915098e+00,  1.15584862e+00,  3.96363706e-01]])

In [75]:
X.columns

Index(['price_EURSEK=X', 'timestamp', 'price_log_EURSEK=X',
       'price_log_diff_EURSEK=X', 'price_CL=F', 'price_log_CL=F',
       'price_log_diff_CL=F', 'price_EUR=X', 'price_log_EUR=X',
       'price_log_diff_EUR=X', 'price_ZB=F', 'price_log_ZB=F',
       'price_log_diff_ZB=F', 'price_GC=F', 'price_log_GC=F',
       'price_log_diff_GC=F', 'price_HG=F', 'price_log_HG=F',
       'price_log_diff_HG=F', 'price_ZS=F', 'price_log_ZS=F',
       'price_log_diff_ZS=F', 'price_SEK=X', 'price_log_SEK=X',
       'price_log_diff_SEK=X', 'price_USDEUR=X', 'price_log_USDEUR=X',
       'price_log_diff_USDEUR=X', 'price_LE=F', 'price_log_LE=F',
       'price_log_diff_LE=F', 'price_SB=F', 'price_log_SB=F',
       'price_log_diff_SB=F', 'price_SI=F', 'price_log_SI=F',
       'price_log_diff_SI=F', 'price', 'price_log', 'price_log_diff'],
      dtype='object')

In [76]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import svm
from sklearn.gaussian_process import GaussianProcessRegressor
from tempfile import mkdtemp
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel


cachedir = mkdtemp()
full_pipeline = Pipeline([
    ('transform', column_pipeline),
    ('pca', PCA()),
    ('regress', 'passthrough')
], memory=cachedir)

N_COMPONENTS = (1, len(num_attribs))
N_FEATURES_OPTIONS = [2,4,8]
GAMMA_OPTIONS = [0.5, 1]
N_ITER=32
N_ITER_SMALL=4

param_grid = [({
    "pca__n_components": N_COMPONENTS,
    "regress": [RandomForestRegressor()],
    "regress__n_estimators":  (10, 100),
    "regress__criterion": ["squared_error"],
    "regress__max_depth": (10,20),
    "regress__min_samples_split": (2,16),
}, N_ITER),
({
    "pca__n_components": N_COMPONENTS,
    "regress": [XGBRegressor()],
    "regress__gamma": (0.5, 5),
    "regress__subsample": (0.6, 1.0),
    'regress__min_child_weight': (1, 10),
    "regress__colsample_bytree": (0.6, 1.0),
    "regress__max_depth": (3, 5)
}, N_ITER),
({
    "pca__n_components": N_COMPONENTS,
    "regress": [svm.SVR()],
    "regress__C": (1,10),
    "regress__gamma": ["scale", "auto"],
    "regress__kernel": ["linear", "poly", "rbf", "sigmoid"]
}, N_ITER),
({
    "pca__n_components": N_COMPONENTS,
    "regress": [lgb.LGBMRegressor()],
    "regress__num_leaves": (31, 127),
    "regress__reg_alpha": [0.1, 0.5],
    "regress__min_data_in_leaf": [30, 50, 100, 300, 400],
    "regress__lambda_l1": [0, 1, 1.5],
    "regress__lambda_l2": [0, 1]
}, N_ITER)]

In [77]:
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV
import os
from skopt import dump, load

def cb(v):
    print('test')
    print(v)

PKL_FILENAME='./grid.pkl'
#grid = GridSearchCV(full_pipeline, n_jobs=1, cv=3, scoring="accuracy", param_grid=param_grid)
if os.path.exists(PKL_FILENAME):
    grid = load(PKL_FILENAME)
else:
    grid =  BayesSearchCV(full_pipeline, n_jobs=2, cv=5, scoring="neg_root_mean_squared_error", search_spaces=param_grid, verbose=3)
    grid.fit(X, y)
    dump(grid, 'grid.pkl')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/home/ken/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/ken/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/ken/.local/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 

In [69]:

# Only use price_log, price_log_diff, got -0.027 score.
# LGBMRegressor(lambda_l1=1.0, lambda_l2=1, min_data_in_leaf=100, num_leaves=65,
#              reg_alpha=0.19838982990813173)
# PCA(n_components=1)

#0.04 score:
#Index(['price_EURSEK=X', 'timestamp', 'price_log_EURSEK=X',
    #    'price_log_diff_EURSEK=X', 'price_CL=F', 'price_log_CL=F',
    #    'price_log_diff_CL=F', 'price_EUR=X', 'price_log_EUR=X',
    #    'price_log_diff_EUR=X', 'price_ZB=F', 'price_log_ZB=F',
    #    'price_log_diff_ZB=F', 'price_GC=F', 'price_log_GC=F',
    #    'price_log_diff_GC=F', 'price_HG=F', 'price_log_HG=F',
    #    'price_log_diff_HG=F', 'price_ZS=F', 'price_log_ZS=F',
    #    'price_log_diff_ZS=F', 'price_SEK=X', 'price_log_SEK=X',
    #    'price_log_diff_SEK=X', 'price_USDEUR=X', 'price_log_USDEUR=X',
    #    'price_log_diff_USDEUR=X', 'price_LE=F', 'price_log_LE=F',
    #    'price_log_diff_LE=F', 'price_SB=F', 'price_log_SB=F',
    #    'price_log_diff_SB=F', 'price_SI=F', 'price_log_SI=F',
    #    'price_log_diff_SI=F', 'price', 'price_log', 'price_log_diff'],
    #   dtype='object')
    # PCA(n_components=6)
    # XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
    #          colsample_bynode=1, colsample_bytree=1.0, enable_categorical=False,
    #          gamma=4.868859599627096, gpu_id=-1, importance_type=None,
    #          interaction_constraints='', learning_rate=0.300000012,
    #          max_delta_step=0, max_depth=3, min_child_weight=4, missing=nan,
    #          monotone_constraints='()', n_estimators=100, n_jobs=4,
    #          num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
    #          reg_lambda=1, scale_pos_weight=1, subsample=0.971239091414603,
    #          tree_method='exact', validate_parameters=1, verbosity=None)





pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__n_components,param_regress,param_regress__criterion,param_regress__max_depth,param_regress__min_samples_split,param_regress__n_estimators,...,param_regress__reg_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
31,2.451594,0.813478,0.013098,0.005613,1,"XGBRegressor(base_score=None, booster=None, co...",NaN,5,NaN,NaN,...,NaN,"{'pca__n_components': 1, 'regress': XGBRegress...",-0.170631,-0.055433,-0.057480,-0.048963,-0.075732,-0.081648,0.045371,1
29,2.368885,0.977233,0.008784,0.004648,1,"XGBRegressor(base_score=None, booster=None, co...",NaN,5,NaN,NaN,...,NaN,"{'pca__n_components': 1, 'regress': XGBRegress...",-0.170631,-0.055433,-0.057480,-0.048963,-0.075732,-0.081648,0.045371,1
18,2.429779,0.782140,0.011424,0.004722,1,"XGBRegressor(base_score=None, booster=None, co...",NaN,5,NaN,NaN,...,NaN,"{'pca__n_components': 1, 'regress': XGBRegress...",-0.170630,-0.055439,-0.057480,-0.048967,-0.075731,-0.081649,0.045369,3
46,48.298334,16.267356,0.020326,0.002548,1,SVR(),NaN,NaN,NaN,NaN,...,NaN,"{'pca__n_components': 1, 'regress': SVR(), 're...",-0.170557,-0.055840,-0.057488,-0.048618,-0.076167,-0.081734,0.045333,4
22,1.980916,0.731604,0.007629,0.000477,2,"XGBRegressor(base_score=None, booster=None, co...",NaN,3,NaN,NaN,...,NaN,"{'pca__n_components': 2, 'regress': XGBRegress...",-0.170629,-0.055278,-0.057480,-0.049445,-0.075881,-0.081743,0.045315,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,0.094430,0.015190,0.007601,0.002126,1,RandomForestRegressor(),squared_error,13,15,12,...,NaN,"{'pca__n_components': 1, 'regress': RandomFore...",-0.174032,-0.056199,-0.116791,-0.060309,-0.080108,-0.097488,0.043865,60
9,0.660827,0.152764,0.023500,0.002146,1,RandomForestRegressor(),squared_error,19,14,84,...,NaN,"{'pca__n_components': 1, 'regress': RandomFore...",-0.175591,-0.062618,-0.106544,-0.064492,-0.084072,-0.098663,0.041617,61
7,0.693683,0.128610,0.024006,0.001729,1,RandomForestRegressor(),squared_error,16,7,91,...,NaN,"{'pca__n_components': 1, 'regress': RandomFore...",-0.176481,-0.060357,-0.120993,-0.065038,-0.083624,-0.101299,0.043229,62
1,0.530621,0.085978,0.025379,0.016423,1,RandomForestRegressor(),squared_error,17,3,63,...,NaN,"{'pca__n_components': 1, 'regress': RandomFore...",-0.177760,-0.063869,-0.117872,-0.068471,-0.085476,-0.102690,0.042048,63


In [55]:
df['value'].std()

0.06310011023976497

In [57]:
grid.best_estimator_

Pipeline(memory='/tmp/tmpzblfpxrr',
         steps=[('transform',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['price_EURSEK=X',
                                                   'price_log_EURSEK=X',
                                                   'price_log_diff_EURSEK=X',
                                                   'price_CL=F',
                                                   'price_log_CL=F',
                                                   'price_log_diff_CL=F',
                                                   'price_EUR=X',
                                                   'price_log_EUR=X',
                                                   'price_log_diff_EUR=X',
                                                   'price...
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=3, min_child_weight=4, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=4, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, subsample=0.971239091414603,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])